In [ ]:
import os
if os.getcwd().split('/')[-1] == 'notebooks':
    os.chdir(os.pardir)

import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
plt.rcParams.update({'text.usetex': True,
                     'text.latex.preamble': r'\usepackage{amsmath}',
                     'font.family': 'serif'})
import numpy as np
np.random.seed(0)
import pandas as pd
import seaborn as sns
sns.set(style='ticks', font='serif')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from smt.applications import MOE
from smt.surrogate_models import QP, RMTB, RMTC

# Training data

In [ ]:
PATH_PROC_DATA = os.path.join('data', 'processed')
syn_data = pd.read_csv(os.path.join(PATH_PROC_DATA, 'pDeltaT_synthetic.csv'))

features = ['d [mm]', 'f [GHz]', 'psPDtot_1 [W/m2]', 'psPDtot_4 [W/m2]']
target = 'pDeltaT * 100 [°C]'
X = syn_data[features]
y = syn_data[target] / 100

X_train, X_test, y_train, y_test = train_test_split(X.to_numpy(),
                                                    y.to_numpy()[:, np.newaxis],
                                                    test_size=.2)

# Mixture of experts

In [ ]:
experts = ['QP', 'RMTB', 'RMTC']
moe = MOE(n_clusters=2,
          smooth_recombination=True,
          allow=experts)
moe.set_training_values(X_train, y_train)
moe.train()

# Evaluating mixture of experts predicting power

In [ ]:
true_data = pd.read_csv(os.path.join(PATH_PROC_DATA, 'pDeltaT_clean.csv'))

X_eval = true_data[features].to_numpy()
y_eval = true_data[target].to_numpy() / 100

y_pred = moe.predict_values(X_eval)
rmse = np.sqrt(mean_squared_error(y_eval, y_pred))

y_resid = (y_eval - y_pred.ravel())
ae = np.abs(y_resid)
mae = np.mean(ae)

In [ ]:
cs = sns.color_palette('rocket', 2)

fig, ax = plt.subplots(1, 1, figsize=(4.5, 4))
ax = sns.histplot(x=ae,
                  color=cs[0], stat='density', kde=True, ax=ax,
                  line_kws={'lw': 2})
ax.vlines(mae, *ax.get_ybound(), colors=cs[1], ls='--', lw=2,
          label='mean absolute error')
ax.legend()
ax.set(xlabel='', ylabel='',
       xticks=[0, mae, 0.25, 0.5],
       xticklabels=[0, round(mae, 2), 0.25, 0.5],
       xlim=[0, 0.5],
       yticks=[0, 4, 8],
       yticklabels=[0, 4, 8],
       ylim=[0, 8]
      )

fig.supxlabel('absolute error [°C]')
fig.supylabel('probability density')
fig.tight_layout()
sns.despine()

# fig_name = os.path.join('figures', '04_1_ape_dist.png')
# fig.savefig(fig_name, dpi=200, bbox_inches='tight')